In [36]:
import pandas as pd
import os
import re
from dataclasses import dataclass
import json
%matplotlib inline

In [37]:
jobname_regex = re.compile(r"(?<=: \[)(?P<project>[a-z-]+), (?P<noise_type>[a-z-]+)")

In [38]:
test_string = "Test Projects: [shiki, baseline]"

res = jobname_regex.search(test_string)

In [39]:
res.group("project")

'shiki'

In [40]:
res.group("noise_type")

'baseline'

# Read in all Results

In [41]:
directory = "../artifacts"
jobs_dir = os.listdir(directory)
f"{len(jobs_dir)} jobs found"

'85 jobs found'

In [42]:
@dataclass
class TestResult:
    project: str
    noise_type: str
    dir: str
    test_results: dict[str, dict[str, int]]
    env_vars: dict[str, str]

    test_df: pd.DataFrame = None

    def __post_init__(self):
        # Compute the test_df
        # Columns: test_name, PASSED, Failed, Skipped, TOTAL RUNS
        test_names = self.test_results.keys()
        passed = [test.get("PASSED", 0) for test in self.test_results.values()]
        failed = [test.get("FAILED", 0) for test in self.test_results.values()]
        skipped = [test.get("SKIPPED", 0) for test in self.test_results.values()]

        self.test_df = pd.DataFrame({
            "test_name": test_names,
            "PASSED": passed,
            "FAILED": failed,
            "SKIPPED": skipped
        })
        self.test_df["TOTAL RUNS"] = self.test_df["PASSED"] + self.test_df["FAILED"] + self.test_df["SKIPPED"]


In [43]:
results: list[TestResult] = []
for job in jobs_dir:
    with open(os.path.join(directory, job, "jobname.txt")) as f:
        job_str = f.read()
    job_re = jobname_regex.search(job_str)

    with open(os.path.join(directory, job, "aggregate-test-results", "aggregated_test_results.json")) as f:
        test_results = json.load(f)

    with open(os.path.join(directory, job, "aggregate-test-results", "environment_variables.json")) as f:
        env_vars = json.load(f)

    results.append(TestResult(
        project=job_re.group("project"),
        noise_type=job_re.group("noise_type"),
        dir=os.path.join(directory, job),
        test_results=test_results,
        env_vars=env_vars
    ))

In [44]:
f"parsed {len(results)} results"

'parsed 85 results'

In [63]:
x =results[0].test_df.sort_values("FAILED")

In [65]:
x

,test_name,PASSED,FAILED,SKIPPED,TOTAL RUNS
0,Default/prerelease/basic-interactions.test.ts/...,58,0,0,58
51,Default/smoke/app.test.ts/can cancel requests,58,0,0,58
50,Default/smoke/app.test.ts/can send requests,58,0,0,58
49,Default/prerelease/request-pane-tab.test.ts/WS...,58,0,0,58
48,Default/prerelease/request-pane-tab.test.ts/WS...,58,0,0,58
...,...,...,...,...,...
19,Default/prerelease/debug-sidebar-interactions....,58,0,0,58
25,Default/prerelease/environment-editor-interact...,58,0,0,58
27,Default/prerelease/git-sync-interactions.test....,0,58,0,58
28,Default/prerelease/git-sync-interactions.test....,0,58,0,58
